In [0]:
from pyspark.sql.functions import lit

In [0]:
asset_df = spark.read.table('db_processed.assets')

In [0]:
daily_df = (asset_df.select(asset_df.exchange_id, 
                             asset_df.base_asset, 
                             asset_df.price, 
                             asset_df.change_24h, 
                             asset_df.ingested_at.alias('price_time'))
                             .withColumn("change_1h", lit('0').cast('float'))
           )

In [0]:
daily_df.printSchema()

root
-- exchange_id: string (nullable = true)
-- base_asset: string (nullable = true)
-- price: float (nullable = true)
-- change_24h: float (nullable = true)
-- price_time: timestamp (nullable = true)
-- change_1h: float (nullable = true)

In [0]:
daily_df.write.mode("overwrite").format("parquet").saveAsTable("db_presentation.daily")

In [0]:
%sql
CONVERT to DELTA db_presentation.daily

In [0]:
daily_summary_df = (asset_df.select(asset_df.exchange_id, 
                                    asset_df.base_asset, 
                                    asset_df.price.alias('current_price'), 
                                    asset_df.ingested_at.alias('price_time'),
                                    asset_df.change_24h.alias('change_1h'),
                                    asset_df.change_24h,
                                    asset_df.price.alias('min_price'),
                                    asset_df.ingested_at.alias('min_time'),
                                    asset_df.price.alias('max_price'),
                                    asset_df.ingested_at.alias('max_time')
                                   )
           )

In [0]:
daily_summary_df.printSchema()

root
-- exchange_id: string (nullable = true)
-- base_asset: string (nullable = true)
-- current_price: float (nullable = true)
-- price_time: timestamp (nullable = true)
-- change_1h: float (nullable = true)
-- change_24h: float (nullable = true)
-- min_price: float (nullable = true)
-- min_time: timestamp (nullable = true)
-- max_price: float (nullable = true)
-- max_time: timestamp (nullable = true)

In [0]:
daily_summary_df.write.mode("overwrite").format("parquet").saveAsTable("db_presentation.daily_summary")

In [0]:
%sql
CONVERT to DELTA db_presentation.daily_summary